In [6]:
import importlib
import numpy as np
import cv2 as cv
from PIL import Image
import lk_util
import lukas_kanade 
import yaml
import time

importlib.reload(lk_util)
importlib.reload(lukas_kanade)



<module 'lukas_kanade' from '/home/hannahcl/Documents/fagV23/Feature-tracking/optical_flow/lukas_kanade.py'>

In [7]:
# -- Import video frames

cap = cv.VideoCapture('https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4')


In [8]:

# Take first frame 
ret, old_frame = cap.read()
target_image = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

# #Find corners with goodFeaturesToTrack
# feature_params = dict( maxCorners = 100,
#                        qualityLevel = 0.3,
#                        minDistance = 7,
#                        blockSize = 7 )
# initial_feature_detections_positions = cv.goodFeaturesToTrack(target_image, mask = None, **feature_params)
# initial_feature_detections = np.array([[*p] for p in initial_feature_detections_positions.reshape(-1,2)])


#  Or, Use FAST feature detector to detect initial features
fast = cv.FastFeatureDetector_create(threshold=100)
initial_feature_detections = fast.detect(target_image, None)
initial_feature_detections = np.array([p.pt for p in initial_feature_detections])

print(initial_feature_detections)



[[304.  86.]
 [350.  97.]
 [272. 104.]
 [301. 105.]
 [236. 150.]
 [254. 150.]
 [264. 150.]
 [183. 185.]
 [162. 191.]
 [308. 211.]
 [160. 215.]
 [165. 218.]
 [380. 244.]]


In [9]:


# Initialize LucasKanade with configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

lk = lukas_kanade.LucasKanade(config)
lk.target_image = target_image
lk.get_initial_feature_detections(initial_feature_detections) 

time_recodring = []

while(True):

    if lk.i_frame >= lk.n_frames -1:
        break

    ret, frame = cap.read()

    start_time = time.perf_counter()
    lk.find_tracks_in_target_image(frame)
    end_time = time.perf_counter()

    elapsed_time_ms = (end_time - start_time) * 1000
    time_recodring.append(elapsed_time_ms)

    if lk.show_and_save_tracks == True:
        lk.show_and_save_tracked_features()

time_recodring = np.array(time_recodring)
average_time = np.average(time_recodring)       
print(f"Over {lk.i_frame}s, it took {average_time} milliseconds on avarage to track {lk.n_features} features.")

Over 101s, it took 116.63493754437722 milliseconds on avarage to track 2 features.


In [10]:

lk.compare_tracks_to_ground_truth()   

cv.waitKey()
cv.destroyAllWindows()




dogbox arctan
Frame 3 and track 0
gt: [237.51762 150.13889], estimated: [304.75614809  86.51376684], error: [67.23852809 63.62512316]
Frame 3 and track 1
gt: [301.1077 103.7322], estimated: [350.07043632  96.45577447], error: [48.96273632  7.27642553]
Frame 10 and track 0
gt: [238.79219 153.61865], estimated: [304.78664067  89.23738533], error: [65.99445067 64.38126467]
Frame 10 and track 1
gt: [303.31433 105.66835], estimated: [345.77336989  97.0316457 ], error: [42.45903989  8.6367043 ]
Frame 50 and track 0
gt: [244.10954 168.41335], estimated: [311.40946097  94.8889392 ], error: [67.29992097 73.5244108 ]
Frame 50 and track 1
gt: [312.2983  113.05156], estimated: [330.66513963  97.76238642], error: [18.36683963 15.28917358]
Frame 100 and track 0
gt: [254.71289 187.72498], estimated: [326.05456547 106.67406331], error: [71.34167547 81.05091669]
Frame 100 and track 1
gt: [327.3952 124.8906], estimated: [334.34432486 103.56148882], error: [ 6.94912486 21.32911118]
